In [ ]:
!pip install lifetimes

     |████████████████████████████████| 593kB 9.1MB/s 


In [ ]:
import lifetimes
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans

In [ ]:
# Leer CSV:
from google.colab import files
uploaded = files.upload()

Saving data_taxi.csv to data_taxi.csv


In [ ]:
import io
df = pd.read_csv(io.StringIO(uploaded['data_taxi.csv'].decode('latin-1')))
df

,user_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,trip_total
0,867ee706a760e0221499dac80d9a7ac935bc13c5f4c4e6...,2021-01-24 18:15:00 UTC,2021-01-24 18:45:00 UTC,1680.0,12.90,63.25
1,b47c583b142d75b42882975eaab19c6cb98d8268601657...,2021-01-20 00:30:00 UTC,2021-01-20 00:30:00 UTC,600.0,4.10,16.50
2,b19847c6c45cc89f385408086d905e8de9069bd2ac1715...,2021-01-22 13:00:00 UTC,2021-01-22 13:15:00 UTC,840.0,6.90,24.25
3,b47c583b142d75b42882975eaab19c6cb98d8268601657...,2021-01-15 07:45:00 UTC,2021-01-15 09:00:00 UTC,3720.0,54.90,201.00
4,54fda80b28cfa29ae2fcd9bfee10b6ab21f4530e1d883a...,2021-01-18 01:30:00 UTC,2021-01-18 01:30:00 UTC,600.0,0.30,23.50
...,...,...,...,...,...,...
301723,5a9018714632ffaccdd5caa19fbd919a62bba3e0d469fe...,2021-02-21 15:15:00 UTC,2021-02-21 15:30:00 UTC,1260.0,11.80,30.75
301724,2c125f03e3308aebd381fe53ddacf94af981e262ae3625...,2021-02-13 14:30:00 UTC,2021-02-13 15:00:00 UTC,2022.0,10.51,30.75
301725,131107e9f0f51399bdc9e8c2eb94fce500bdf22daeb1f8...,2021-02-25 20:15:00 UTC,2021-02-25 20:45:00 UTC,2100.0,0.00,30.75
301726,50e6045759886233d0e834d9877f52debc185d92adbdae...,2021-02-22 16:15:00 UTC,2021-02-22 16:45:00 UTC,1500.0,11.70,30.75


In [ ]:
miles = df[["user_id", "trip_miles"]]
miles

,user_id,trip_miles
0,867ee706a760e0221499dac80d9a7ac935bc13c5f4c4e6...,12.90
1,b47c583b142d75b42882975eaab19c6cb98d8268601657...,4.10
2,b19847c6c45cc89f385408086d905e8de9069bd2ac1715...,6.90
3,b47c583b142d75b42882975eaab19c6cb98d8268601657...,54.90
4,54fda80b28cfa29ae2fcd9bfee10b6ab21f4530e1d883a...,0.30
...,...,...
301723,5a9018714632ffaccdd5caa19fbd919a62bba3e0d469fe...,11.80
301724,2c125f03e3308aebd381fe53ddacf94af981e262ae3625...,10.51
301725,131107e9f0f51399bdc9e8c2eb94fce500bdf22daeb1f8...,0.00
301726,50e6045759886233d0e834d9877f52debc185d92adbdae...,11.70


In [ ]:
miles.to_excel('LTV_trip_miles.xlsx',index = False, encoding='UTF-8')

In [ ]:
#Tabla resumen de las transacciones:
summary = lifetimes.utils.summary_data_from_transaction_data(df, 'user_id', 'trip_start_timestamp', 'trip_total' )
summary = summary.reset_index()
df_2 = summary
df_2

/usr/local/lib/python3.7/dist-packages/lifetimes/utils.py:287: UserWarning: Converting to Period representation will drop timezone information.
  pd.to_datetime(transactions[datetime_col].max(), format=datetime_format).to_period(freq).to_timestamp()
/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:1092: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


,user_id,frequency,recency,T,monetary_value
0,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,33.0,48.0,49.0,144.770606
1,00752524a4118a162dce62ff49c0eb0f79530d9e0ecb5a...,26.0,58.0,59.0,104.263462
2,008c43f04857eb09949b8ce05458d3fc8848dfd460d925...,3.0,3.0,4.0,162.716667
3,00aeef49089b36ed668920fac1119a26fb25d1b48102a4...,49.0,58.0,59.0,157.688367
4,00c0020de20ee2cd36c1d41058360c24dad77a52639413...,52.0,56.0,57.0,129.044423
...,...,...,...,...,...
844,fe659851fec6b2684321a52e42b1a316dd6657ba108f61...,47.0,56.0,58.0,188.184043
845,feb9a71dcd524378d243372a5a61e3d50553304b130979...,50.0,57.0,58.0,163.139000
846,ff02a85901ca79b5cbceeede05705f16c2b3ddfce6675b...,0.0,0.0,12.0,0.000000
847,ff4e08928c034ee1ff1155d943718f52fa77283f7d8d3b...,51.0,58.0,59.0,204.192745


In [ ]:
#Método de Clustering:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final 

In [ ]:
#Cluster Recencia:
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_2[['recency']])
df_2['RecencyCluster'] = kmeans.predict(df_2[['recency']])
df_2.head()

,user_id,frequency,recency,T,monetary_value,RecencyCluster
0,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,33.0,48.0,49.0,144.770606,3
1,00752524a4118a162dce62ff49c0eb0f79530d9e0ecb5a...,26.0,58.0,59.0,104.263462,0
2,008c43f04857eb09949b8ce05458d3fc8848dfd460d925...,3.0,3.0,4.0,162.716667,2
3,00aeef49089b36ed668920fac1119a26fb25d1b48102a4...,49.0,58.0,59.0,157.688367,0
4,00c0020de20ee2cd36c1d41058360c24dad77a52639413...,52.0,56.0,57.0,129.044423,0


In [ ]:
#Recencia por Cluster:
df_2.groupby('RecencyCluster')['recency'].describe()

,count,mean,std,min,25%,50%,75%,max
RecencyCluster,,,,,,,,
0,617.0,56.721232,1.750720,51.0,56.0,57.0,58.0,59.0
1,67.0,21.865672,5.833278,13.0,17.0,20.0,26.0,32.0
2,72.0,3.138889,3.655232,0.0,0.0,2.0,6.0,12.0
3,93.0,43.892473,4.796879,33.0,40.0,45.0,48.0,50.0


In [ ]:
#Cluster de recencia 
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_2[['frequency']])
df_2['FrequencyCluster'] = kmeans.predict(df_2[['frequency']])
df_2.head()

,user_id,frequency,recency,T,monetary_value,RecencyCluster,FrequencyCluster
0,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,33.0,48.0,49.0,144.770606,3,3
1,00752524a4118a162dce62ff49c0eb0f79530d9e0ecb5a...,26.0,58.0,59.0,104.263462,0,3
2,008c43f04857eb09949b8ce05458d3fc8848dfd460d925...,3.0,3.0,4.0,162.716667,2,1
3,00aeef49089b36ed668920fac1119a26fb25d1b48102a4...,49.0,58.0,59.0,157.688367,0,2
4,00c0020de20ee2cd36c1d41058360c24dad77a52639413...,52.0,56.0,57.0,129.044423,0,2


In [ ]:
#Frecuencia por Cluster:
df_2.groupby('FrequencyCluster')['frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,245.0,42.763265,4.092311,35.0,39.0,43.0,47.0,48.0
1,122.0,4.967213,4.746350,0.0,1.0,4.0,9.0,15.0
2,369.0,54.384824,2.899285,49.0,52.0,55.0,57.0,59.0
3,113.0,26.495575,5.620631,16.0,21.0,27.0,31.0,34.0


In [ ]:
#Cluster para revenue
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_2[['monetary_value']])
df_2['RevenueCluster'] = kmeans.predict(df_2[['monetary_value']])
df_2 = order_cluster('RevenueCluster', 'monetary_value',df_2,True)
df_2.head()

,user_id,frequency,recency,T,monetary_value,RecencyCluster,FrequencyCluster,RevenueCluster
0,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,33.0,48.0,49.0,144.770606,3,3,1
1,008c43f04857eb09949b8ce05458d3fc8848dfd460d925...,3.0,3.0,4.0,162.716667,2,1,1
2,00aeef49089b36ed668920fac1119a26fb25d1b48102a4...,49.0,58.0,59.0,157.688367,0,2,1
3,00c0020de20ee2cd36c1d41058360c24dad77a52639413...,52.0,56.0,57.0,129.044423,0,2,1
4,00c152d49563ae56e208fde25f6c65b08511d0f50259d1...,57.0,58.0,59.0,198.302982,0,2,1


In [ ]:
#LTV por Cluster:
df_2.groupby('RevenueCluster')['monetary_value'].describe()

,count,mean,std,min,25%,50%,75%,max
RevenueCluster,,,,,,,,
0,283.0,72.661825,40.758770,0.000000,40.353835,85.220000,105.733554,124.450408
1,423.0,176.799442,32.308744,125.550909,148.831290,174.017241,201.426183,247.689787
2,140.0,318.597863,72.178345,248.755000,274.394301,302.006747,339.620068,790.700000
3,3.0,1372.163206,388.031867,1058.271600,1155.241059,1252.210517,1529.109009,1806.007500


In [ ]:
#overall scoring
df_2['OverallScore'] = df_2['RecencyCluster'] + df_2['FrequencyCluster'] + df_2['RevenueCluster']
df_2['Segment'] = 'Low-Value'
df_2.loc[df_2['OverallScore']>2,'Segment'] = 'Mid-Value' 
df_2.loc[df_2['OverallScore']>4,'Segment'] = 'High-Value' 
df_2.head()

,user_id,frequency,recency,T,monetary_value,RecencyCluster,FrequencyCluster,RevenueCluster,OverallScore,Segment
0,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,33.0,48.0,49.0,144.770606,3,3,1,7,High-Value
1,008c43f04857eb09949b8ce05458d3fc8848dfd460d925...,3.0,3.0,4.0,162.716667,2,1,1,4,Mid-Value
2,00aeef49089b36ed668920fac1119a26fb25d1b48102a4...,49.0,58.0,59.0,157.688367,0,2,1,3,Mid-Value
3,00c0020de20ee2cd36c1d41058360c24dad77a52639413...,52.0,56.0,57.0,129.044423,0,2,1,3,Mid-Value
4,00c152d49563ae56e208fde25f6c65b08511d0f50259d1...,57.0,58.0,59.0,198.302982,0,2,1,3,Mid-Value


In [ ]:
#calculate and show correlations
corr_matrix = df_2.corr()
corr_matrix['monetary_value'].sort_values(ascending=False)

monetary_value      1.000000
RevenueCluster      0.786739
OverallScore        0.335429
frequency           0.291316
recency             0.199052
FrequencyCluster    0.085055
T                   0.057034
RecencyCluster     -0.115977
Name: monetary_value, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [ ]:
X=df_2[['OverallScore', 'recency', 'frequency']]
y=df_2[['monetary_value']]

In [ ]:
#Se deividen los datos en parte de pruebas y de entranimiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)

In [ ]:
#Ajuste del modelo
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#Predicciones los datos de prueba:
y_pred = linreg.predict(X_test)
y_pred

array([[181.68216355],
       [200.11503251],
       [ 75.3654113 ],
       [195.81867819],
       [222.15446215],
       [249.16902526],
       [192.80525817],
       [210.53609942],
       [144.98648303],
       [210.53609942],
       [200.74487538],
       [200.12726658],
       [244.27953028],
       [150.97540127],
       [ 75.06710789],
       [217.8703419 ],
       [198.31236196],
       [154.78994114],
       [159.69167019],
       [128.33181648],
       [ 84.84609786],
       [186.57165854],
       [200.72040724],
       [ 75.3654113 ],
       [212.96861285],
       [212.98084691],
       [144.98648303],
       [ 89.73559285],
       [215.94411107],
       [152.33295957],
       [241.83478279],
       [217.8703419 ],
       [123.49125777],
       [108.82277282],
       [ 87.26637721],
       [151.8621596 ],
       [ 35.39470956],
       [208.07911786],
       [200.75710945],
       [ 70.17761291],
       [212.98084691],
       [244.26729621],
       [159.67943612]])

In [ ]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

[-18.53063879]
[[3.12986834e+01 1.22340696e-02 2.44474749e+00]]


In [ ]:
#Que tan bien el modelo ajusta la data.
from sklearn import metrics

# compute the R Square for model
print("R-Square:",metrics.r2_score(y_test, y_pred))


R-Square: 0.38546952214630814


In [ ]:
y_hats = linreg.predict(X_test)
y_hats = pd.DataFrame(y_hats)
#df_class["Prediction"]=y_hats
#df_class.head()
y_hats

,0
0,181.682164
1,200.115033
2,75.365411
3,195.818678
4,222.154462
5,249.169025
6,192.805258
7,210.536099
8,144.986483
9,210.536099


In [ ]:
y_hats = linreg.predict(X_test)
y_hats = pd.DataFrame(y_hats)
#df_class["Prediction"]=y_hats
#df_class.head()
df_2["Prediction"] = linreg.predict(X)
df_2

,user_id,frequency,recency,T,monetary_value,RecencyCluster,FrequencyCluster,RevenueCluster,OverallScore,Segment,Prediction
0,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,33.0,48.0,49.0,144.770606,3,3,1,7,High-Value,281.824047
1,008c43f04857eb09949b8ce05458d3fc8848dfd460d925...,3.0,3.0,4.0,162.716667,2,1,1,4,Mid-Value,114.035039
2,00aeef49089b36ed668920fac1119a26fb25d1b48102a4...,49.0,58.0,59.0,157.688367,0,2,1,3,Mid-Value,195.867614
3,00c0020de20ee2cd36c1d41058360c24dad77a52639413...,52.0,56.0,57.0,129.044423,0,2,1,3,Mid-Value,203.177389
4,00c152d49563ae56e208fde25f6c65b08511d0f50259d1...,57.0,58.0,59.0,198.302982,0,2,1,3,Mid-Value,215.425594
...,...,...,...,...,...,...,...,...,...,...,...
844,faae54dec275b76c21410117883cd1421b2548bc9a54c7...,57.0,57.0,58.0,306.439474,0,2,2,4,Mid-Value,246.712044
845,fc9d2d20856d6ccf05e9ad79233cf3d26a11a334ba2138...,35.0,48.0,49.0,254.292571,3,0,2,5,High-Value,224.116176
846,53de0dbfc9e66a4ae663f3383fbbf41032653d04aa8700...,58.0,58.0,59.0,1252.210517,0,2,3,5,High-Value,280.467709
847,a1b8eb24fa71867de525452474490e1c822269411962fe...,50.0,58.0,59.0,1058.271600,0,2,3,5,High-Value,260.909729


In [ ]:
#Checking missing variable
df_2.isnull().sum().sort_values(ascending=False).head()

Prediction          0
Segment             0
OverallScore        0
RevenueCluster      0
FrequencyCluster    0
dtype: int64

In [ ]:
#Cluster para predicción
kmeans = KMeans(n_clusters=4)
kmeans.fit(df_2[['Prediction']])
df_2['PredictionCluster'] = kmeans.predict(df_2[['Prediction']])
df_2 = order_cluster('PredictionCluster', 'Prediction',df_2,True)
df_2.head()

,user_id,frequency,recency,T,monetary_value,RecencyCluster,FrequencyCluster,RevenueCluster,OverallScore,Segment,Prediction,PredictionCluster
0,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,33.0,48.0,49.0,144.770606,3,3,1,7,High-Value,281.824047,3
1,0309e367d50b97dc7c5d764697737bfac5e6d048f72798...,24.0,43.0,44.0,203.941250,3,3,1,7,High-Value,259.760150,3
2,223702c6155649517dcb9f4860ae0fa2669903bbe7d54f...,32.0,47.0,48.0,212.994375,3,3,1,7,High-Value,279.367066,3
3,2c5a6f3edf1ce6d713e455e58c37609cfdb51023cac7dc...,33.0,48.0,51.0,137.053333,3,3,1,7,High-Value,281.824047,3
4,2ff4fad54ffae9d505e55587c34111c2094a0a52873695...,29.0,37.0,58.0,182.892069,3,3,1,7,High-Value,271.910483,3


In [ ]:
df_2.to_excel('LTV_prediction_pilot.xlsx',index = False, encoding='UTF-8')

In [ ]:
#LTV Prediciton por Cluster:
df_2.groupby('PredictionCluster')['Prediction'].describe()

,count,mean,std,min,25%,50%,75%,max
PredictionCluster,,,,,,,,
0,175.0,81.999264,16.496370,25.627954,75.365411,79.956603,94.618971,109.824324
1,201.0,138.207459,18.437080,111.206350,123.466790,130.825500,154.789941,169.458426
2,353.0,202.138005,13.902610,170.319497,192.695152,205.646604,212.980847,224.623678
3,120.0,249.216885,17.993594,227.141830,239.353333,244.279530,249.169025,315.579712


In [ ]:
linreg.score(X_test, y_test)

0.38546952214630814

In [ ]:
from google.cloud import bigquery

In [ ]:
from google.colab import auth
auth.authenticate_user()
bq = bigquery.Client(project="demos-209421")

In [ ]:
output_table = 'dataset_LTV_final'


## RESULTADOS HACIA BIGQUERY - TABLA FINAL
table_id_final = 'demos-209421.LTV.{0}'.format(output_table)
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("user_id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("frequency", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField("recency", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField("T", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField("monetary_value", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField("RecencyCluster", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("FrequencyCluster", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("OverallScore", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("Segment", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Prediction", bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField("PredictionCluster", bigquery.enums.SqlTypeNames.INTEGER)
    ],
    write_disposition="WRITE_TRUNCATE",)
job = bq.load_table_from_dataframe(df_2, table_id_final, job_config=job_config)  # Make an API request.
job.result()  # Wait for the job to complete.
table = bq.get_table(table_id_final)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id_final
    )
)

Loaded 849 rows and 12 columns to demos-209421.LTV.dataset_LTV_final
